# Timing comparisons with lacosmic and astroscrappy

In this notebook, we compare dfcosmic (both CPU and GPU implementations) with [lacosmic](https://github.com/larrybradley/lacosmic) and [astroscrappy](https://github.com/astropy/astroscrappy). 

These tests were run using a System76 Adder WS with a Intel® Core™ i9-14900HX × 32 and a NVIDIA GeForce RTX 4060 Laptop GPU.

The code snippet for generating fake data was taken **directly** from [astroscrappy tests](https://github.com/astropy/astroscrappy/blob/main/astroscrappy/tests/fake_data.py); therefore, we thank them.

The fake data is set to have the approximate size as a standard narrowband observation taken with Mothra (6000 x 4000).

In [4]:
import sys
!{sys.executable} -m pip install --upgrade numba

Defaulting to user installation because normal site-packages is not writeable
  Using cached numba-0.62.1.tar.gz (2.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [24 lines of output]
      Traceback (most recent call last):
        File "/usr/lib/python3.14/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 389, in <module>
          main()
          ~~~~^^
        File "/usr/lib/python3.14/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ~~~~^^^^^^^^^^^^^^^^^^^^^^^^
        File "/usr/lib/python3.14/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 143, in get_requires_for_build_wheel
          return hook(config_settings)

In [3]:
import time
import numpy as np
from lacosmic.core import lacosmic
from astroscrappy import detect_cosmics
from dfcosmic import lacosmic as df_lacosmic

In [2]:
# Make a simple Gaussian function for testing purposes
def gaussian(image_shape, x0, y0, brightness, fwhm):
    x = np.arange(image_shape[1])
    y = np.arange(image_shape[0])
    x2d, y2d = np.meshgrid(x, y)

    sig = fwhm / 2.35482

    normfactor = brightness / 2.0 / np.pi * sig**-2.0
    exponent = -0.5 * sig**-2.0
    exponent *= (x2d - x0) ** 2.0 + (y2d - y0) ** 2.0

    return normfactor * np.exp(exponent)


def make_fake_data(size=(6000, 4000)):
    """
    Generate fake data that can be used to test the detection and cleaning algorithms

    Returns
    -------
    imdata : numpy float array
        Fake Image data
    crmask : numpy boolean array
        Boolean mask of locations of injected cosmic rays
    """
    # Set a seed so that the tests are repeatable
    np.random.seed(200)

    # Create a simulated image to use in our tests
    imdata = np.zeros(size, dtype=np.float32)

    # Add sky and sky noise
    imdata += 200

    psf_sigma = 3.5

    # Add some fake sources
    for i in range(100):
        x = np.random.uniform(low=0.0, high=1001)
        y = np.random.uniform(low=0.0, high=1001)
        brightness = np.random.uniform(low=1000.0, high=30000.0)
        imdata += gaussian(imdata.shape, x, y, brightness, psf_sigma)

    # Add the poisson noise
    imdata = np.float32(np.random.poisson(imdata))

    # Add readnoise
    imdata += np.random.normal(0.0, 10.0, size=size)

    # Add 100 fake cosmic rays
    cr_x = np.random.randint(low=5, high=995, size=100)
    cr_y = np.random.randint(low=5, high=995, size=100)

    cr_brightnesses = np.random.uniform(low=1000.0, high=30000.0, size=100)

    imdata[cr_y, cr_x] += cr_brightnesses
    imdata = imdata.astype("f4")

    # Make a mask where the detected cosmic rays should be
    crmask = np.zeros(size, dtype=bool)
    crmask[cr_y, cr_x] = True
    return imdata, crmask

In [3]:
# Make fake data
imdata, expected_crmask = make_fake_data()

Run the code using our four options:

1. dfcosmic on the CPU
2. dfcosmic on the GPU
3. astroscrappy
4. lacosmic

In [4]:
start_dfcosmic_cpu = time.perf_counter()
clean, crmask = df_lacosmic(
    image=imdata,
    objlim=2,
    sigfrac=1,
    sigclip=6,
    gain=1,
    readnoise=10,
    niter=1,
    device="cpu",
)

elapsed_dfcosmic_cpu = time.perf_counter() - start_dfcosmic_cpu

/home/carterrhea/Documents/dfcosmic/src/dfcosmic/utils.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  block_size_int = torch.tensor(block_size, dtype=torch.int)


In [5]:
start_dfcosmic_gpu = time.perf_counter()
clean, crmask = df_lacosmic(
    image=imdata,
    objlim=2,
    sigfrac=1,
    sigclip=6,
    gain=1,
    readnoise=10,
    niter=1,
    device="cuda",
)

elapsed_dfcosmic_gpu = time.perf_counter() - start_dfcosmic_gpu

In [6]:
!python -m pip install numba --upgrade

In [7]:
from dfcosmic.utils import (
    avg_pool2d_numpy_fast,
    block_replicate_scipy,
    block_replicate_torch,
    convolve,
    dilation_pytorch,
    median_filter_torch,
    sigma_clip_pytorch,
)


import numpy as np
from scipy.ndimage import convolve, binary_dilation
import numba

# ----------------------------
# Numba helpers
# ----------------------------

@numba.njit
def median_filter_2d(image, kernel_size):
    """
    2D median filter, square kernel, single-threaded.
    """
    pad = kernel_size // 2
    H, W = image.shape
    padded = np.pad(image, pad, mode='edge')
    out = np.empty_like(image)
    
    for i in range(H):
        for j in range(W):
            window = padded[i:i+kernel_size, j:j+kernel_size]
            out[i, j] = np.median(window)
    return out

@numba.njit
def block_average(image, block_size):
    """
    Downsample by block averaging.
    """
    bh, bw = block_size
    H, W = image.shape
    Ht = (H // bh) * bh
    Wt = (W // bw) * bw
    out = np.empty((Ht//bh, Wt//bw), dtype=image.dtype)
    
    for i in range(Ht//bh):
        for j in range(Wt//bw):
            block = image[i*bh:(i+1)*bh, j*bw:(j+1)*bw]
            out[i,j] = np.mean(block)
    return out

# ----------------------------
# Main LA Cosmic function
# ----------------------------

def df_lacosmic_scipy(
    image: np.ndarray,
    sigclip: float = 4.5,
    sigfrac: float = 0.5,
    objlim: float = 1.0,
    niter: int = 1,
    gain: float = 0.0,
    readnoise: float = 0.0,
) -> tuple[np.ndarray, np.ndarray]:
    """
    Optimized single-threaded LA Cosmic using Numba for median filtering and block averaging.
    """
    # Kernels
    block_size_tuple = (2, 2)
    laplacian_kernel = np.array([[0, -1, 0],
                                 [-1, 4, -1],
                                 [0, -1, 0]])
    strel = np.ones((3,3))

    clean_image = image.copy()
    final_crmask = np.zeros(clean_image.shape, dtype=bool)

    for iteration in range(niter):
        # Step 0: gain estimate if not provided
        if gain <= 0:
            sky_level = sigmaclip(clean_image, low=5, high=5)
            med7 = median_filter_2d(clean_image, 7)
            residuals = clean_image - med7
            abs_residuals = np.abs(residuals)
            mad = sigmaclip(abs_residuals, low=5, high=5)
            sig = 1.48 * mad
            if sig == 0:
                raise ValueError(f"Gain determination failed. Sky: {sky_level:.2f}, Sigma: {sig:.2f}")
            gain = sky_level / (sig**2)
            if gain <= 0:
                raise ValueError(f"Gain determination failed. Sky: {sky_level:.2f}, Sigma: {sig:.2f}")

        # Step 1: Laplacian detection
        temp = block_replicate_scipy(clean_image, block_size_tuple)
        temp = convolve(temp, laplacian_kernel, mode='constant', cval=0.0)
        temp = np.clip(temp, 0, None)
        temp = block_average(temp, block_size_tuple)  # downsample

        # Step 2: Noise model
        noise_model = median_filter_2d(clean_image, 5)
        noise_model = np.clip(noise_model, 1e-5, None)
        noise_model = np.sqrt(noise_model * gain + readnoise**2) / gain

        # Step 3: Significance map
        sig_map = temp / noise_model
        sig_map /= 2
        sig_map -= median_filter_2d(sig_map, 5)

        # Step 4: Initial cosmic ray candidates
        cr_mask = (sig_map > sigclip).astype(np.float32)

        # Step 5: Reject objects
        temp = median_filter_2d(clean_image, 3)
        temp -= median_filter_2d(temp, 7)
        temp /= noise_model
        temp = np.clip(temp, 0.01, None)
        cr_mask *= ((sig_map / temp) > objlim).astype(np.float32)

        # Step 6: Neighbor pixel rejection
        sigcliplow = sigclip * sigfrac

        # First growth
        cr_mask = binary_dilation(cr_mask > 0, structure=strel).astype(np.float32)
        cr_mask = (cr_mask * sig_map > sigclip).astype(np.float32)

        # Second growth
        cr_mask = binary_dilation(cr_mask > 0, structure=strel).astype(np.float32)
        cr_mask = (cr_mask * sig_map > sigcliplow).astype(np.float32)

        # Step 7: Update final mask & clean image
        final_crmask |= cr_mask.astype(bool)
        temp = clean_image.astype(np.float32)
        temp[final_crmask] = -9999
        temp = median_filter_2d(temp, 5)
        clean_image[final_crmask] = temp[final_crmask]

    return clean_image, final_crmask



from scipy.ndimage import zoom

def block_replicate_scipy(
    data: np.ndarray, block_size: tuple[int, int], conserve_sum: bool = True
):
    out = zoom(data, zoom=block_size, order=0)  # order=0 → nearest neighbor

    if conserve_sum:
        out = out / np.prod(block_size)
    return out

ModuleNotFoundError: No module named 'numba'

In [ ]:
start_dfcosmic_scipy = time.perf_counter()
clean, crmask = df_lacosmic_scipy(
    image=imdata,
    objlim=2,
    sigfrac=1,
    sigclip=6,
    gain=1,
    readnoise=10,
    niter=1,
)

elapsed_dfcosmic_scipy = time.perf_counter() - start_dfcosmic_scipy

print(elapsed_dfcosmic_scipy)

In [35]:
start_astroscrappy = time.perf_counter()
mask, _clean = detect_cosmics(imdata, readnoise=10.0, gain=1.0, sigclip=6, sigfrac=1.0)

elapsed_astroscrappy = time.perf_counter() - start_astroscrappy

In [36]:
start_lacosmic = time.perf_counter()
clean, crmask = lacosmic(
    data=imdata,
    contrast=2,
    neighbor_threshold=1,
    cr_threshold=6,
    effective_gain=1,
    readnoise=10,
    maxiter=1,
)

elapsed_lacosmic = time.perf_counter() - start_lacosmic

INFO: Iteration 1: Found 162 cosmic-ray pixels, Total: 162 [lacosmic.core]


Print out runtimes

In [40]:
print(f"dfcosmic CPU runtime: {elapsed_dfcosmic_cpu:.4f} seconds")
#print(f"dfcosmic GPU runtime: {elapsed_dfcosmic_gpu:.4f} seconds")
print(f"dfcosmic scipy runtime: {elapsed_dfcosmic_scipy:.4f} seconds")
print(f"astroscrappy runtime: {elapsed_astroscrappy:.4f} seconds")
print(f"lacosmic runtime: {elapsed_lacosmic:.4f} seconds")

dfcosmic CPU runtime: 22.7335 seconds
dfcosmic scipy runtime: 48.5994 seconds
astroscrappy runtime: 5.5399 seconds
lacosmic runtime: 39.9137 seconds
